In [ ]:
import os
import json
import glob
import torch
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
#import seaborn as sns
import matplotlib.pyplot as plt
from utils.data_processing import (
    load_edge_scores_into_dictionary,
    read_json_file,
    get_ckpts,
    load_metrics,
    compute_ged,
    compute_weighted_ged,
    compute_gtd,
    compute_jaccard_similarity_to_reference,
    compute_jaccard_similarity,
    compute_weighted_jaccard_similarity,
    compute_weighted_jaccard_similarity_to_reference,
    aggregate_metrics_to_tensors_step_number,
    get_ckpts
)

from utils.visualization import plot_graph_metric

In [ ]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
MODEL_NAME = 'pythia-160m'

In [ ]:
folder_path = f'results/graphs/{MODEL_NAME}/{TASK}'
df = load_edge_scores_into_dictionary(folder_path)
df = df[df['checkpoint'] >= 4000]

perf_metrics = torch.load(f'results/backup/{MODEL_NAME}/nmh_backup_metrics.pt')
perf_metric_dict = {checkpoint: perf_metrics[checkpoint]['logit_diff'] for checkpoint in perf_metrics.keys()}

## Graph Metrics

In [ ]:
# Summarize the total number of edges in the graph at each checkpoint, filtered by in_circuit
subgraph_df = df[df['in_circuit'] == True]

# Group by checkpoint and sum the number of edges
subgraph_df = subgraph_df.groupby('checkpoint').size().reset_index(name='num_edges')

subgraph_df.head()

### Graph Size

In [ ]:
plot_graph_metric(subgraph_df, 'num_edges', perf_metric_dict, f'Graph Size for {MODEL_NAME}', y_range=1000, x_axis_col='checkpoint', log_x=True)

plot_graph_metric(model_df, 'in_circuit', perf_metric_dict, f'Graph Size for {model}', y_range=1000, x_axis_col='checkpoint', log_x=True)